In [2]:
#%%
#%tensorflow_version 1.x
import tensorflow as tf
#import keras
print("Using tensorflow version " + str(tf.__version__))
#print("Using keras version " + str(keras.__version__))

import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import math
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

Using tensorflow version 2.2.0-rc2


In [3]:
#%%
# Load training data

nrows_train = 1068504
nrows_test = 0

from google.colab import drive
drive.mount('/content/gdrive')
root_dir = "/content/gdrive/My Drive/"

xtrain = np.loadtxt(root_dir + 'xtrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_test)
ytrain = np.loadtxt(root_dir + 'ytrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_test)
ytrain = np.array(ytrain).reshape(nrows_train + nrows_test)
# Check the number of observations and properties
#print(xtrain[:3,])
#print(ytrain[:10])
#print(xtrain.shape)
#print(ytrain.shape)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In the cell below, we perform data augmentation techniques.

Our algorithm (decision trees) doesn't require scaled data, and allow their range to be heterogeneous. However, we can still mirror the data: since each sample is composed of two faces, and a certain range of features is separable with respect to one of the two faces, we can simply exchange them. This increases the size of the training set by a 2 factor.

In [4]:
#%%
# Pre-processing: we just remove the 13*2 first features, concerning only one of the two faces
#xtrain_preprocessed = xtrain[:, 26:]
xtrain = xtrain.astype('float32')
xtest = np.loadtxt(root_dir + 'xtest_challenge.csv', delimiter=',', skiprows = 1).astype('float32')

# We change the columns
#l2_diff_train = np.sqrt((xtrain[:, 0:13] - xtrain[:, 13:26])**2)
#l2_diff_test = np.sqrt((xtest[:, 0:13] - xtest[:, 13:26])**2)
#xtrain = np.hstack((l2_diff_train, xtrain[:, 26:]))
#xtest = np.hstack((l2_diff_test, xtest[:, 26:]))

scaler = StandardScaler()
scaler.fit(xtrain)

x_train_permuter = np.copy(xtrain)
x_train_permuter[:, :13] = xtrain[:, 13:26]
x_train_permuter[:, 13:26] = xtrain[:, :13]

new_x_train = np.concatenate((xtrain, x_train_permuter), axis=0)
new_y_train = np.concatenate((ytrain, ytrain), axis=0)

#xtrain_preprocessed=scaler.transform(xtrain)
xtrain_preprocessed = xtrain
#xtest_preprocessed = scaler.transform(xtest)
xtest_preprocessed = xtest
#xtrain_preprocessed, xval_preprocessed, ytrain, yval = train_test_split(xtrain_preprocessed, ytrain, test_size=0.0) #KAMIKAZE
print(xtrain_preprocessed.shape)
print(ytrain.shape)

(1068504, 37)
(1068504,)


In the code cell below we apply cross-validation on the XGBClassifier model. We boost the learning of many decision trees which are considered as weak leaners.
The most important parameters are :
- learning_rate: similar to other gradient descent methods, the classifier perform more or less a gradient correction. A small learning rate will lead to very slow convergence, a big learning rate can make us miss the sweet spot.
- n_estimators: to be considered as a trade-off with the learning rate. It is the number of rounds performed by the classifier during learning. A too great number will lead to over-fitting.
- max_depth: the maximum depth related to the decision trees. The less it is, the more the decision trees are basic (hence having higher bias, which is not a problem for boosting).

In [13]:
from xgboost import XGBClassifier

with tf.device('/device:GPU:0'):
  boost = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,max_depth=6
                        gamma=0, learning_rate=0.25, max_delta_step=0,
                        min_child_weight=1, missing=None, n_estimators=700, nthread=-1,
                        objective='binary:logistic', sampling_method='gradient_based',
                        reg_alpha=0, reg_lambda=1, tree_method='gpu_hist',
                        scale_pos_weight=1, seed=0, silent=True, subsample=1)
  
  #p_grid_boost = {'max_depth': [2,3,4,5,6,7,8,9,10], 'n_estimators':[500,600,700,800,900,1000]}   

  #grid_boost = GridSearchCV(estimator=boost, param_grid=p_grid_boost, scoring="accuracy", cv=5)
  
  #grid_boost.fit(xtrain_preprocessed, ytrain)
  boost.fit(new_x_train, new_y_train)

  #print("Best Score: {}".format(grid_boost.best_score_))
  #print("Best params: {}".format(grid_boost.best_params_))
  #p_boost = boost.predict(xval_preprocessed)
  #print(p_boost.shape)

Best Score: 0.9990847015822186
Best params: {'max_depth': 6, 'n_estimators': 700}


In [7]:
#boost = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                        gamma=0, learning_rate=0.25, max_delta_step=0, max_depth=6,
                        min_child_weight=1, missing=None, n_estimators=700, nthread=-1,
                        objective='binary:logistic', sampling_method='gradient_based',
                        reg_alpha=0, reg_lambda=1, tree_method='gpu_hist',
                        scale_pos_weight=1, seed=0, silent=True, subsample=1)
#boost.fit(new_x_train, new_y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.25, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=700, n_jobs=1,
              nthread=-1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, sampling_method='gradient_based',
              scale_pos_weight=1, seed=0, silent=True, subsample=1,
              tree_method='gpu_hist', verbosity=1)

In [0]:
output = boost.predict(xtest_preprocessed)

In [0]:
np.savetxt(root_dir + 'ytest_challenge_student_cv3.csv', output, fmt = '%1.0d', delimiter=',')

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout

# START CODE HERE
input_shape = 24
output_shape = 2

new_model = Sequential([
    Dense(24, activation='relu', input_dim=(input_shape)),
    Dense(10, activation='relu'),
    Dense(6, activation='relu'),
    Dense(2, activation='softmax')
])

In [0]:
# The optimizers module provides a number of optimization algorithms for updating
# a netwok parameters accoridng to the computed error gradints
from keras import optimizers

# START CODE HERE

sgd = keras.optimizers.SGD(lr=0.08)

new_model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])
# END CODE HERE
# We can now have a look at the defined model topology
new_model.summary()

In [0]:
# This is where the actual training-testing happens
# Number of epochs we want to train
n_epochs = 2

# START CODE HERE
history = new_model.fit(xtrain_preprocessed, ytrain, batch_size=32, epochs=n_epochs, verbose=1, validation_data=(xval_preprocessed, yval))
# END CODE HERE

In [0]:
pred = new_model.predict_classes(xtest_preprocessed)

In [0]:
print(pred.shape)
output = pred #ok
np.savetxt(root_dir + 'ytest_challenge_student.csv', output, fmt = '%1.0d', delimiter=',')